In [42]:

# 데이터 다운로드
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv -O ChatbotData.csv

# 판다스로 확인
import pandas as pd

data = pd.read_csv('ChatbotData.csv')
print("총 샘플 수:", len(data))
data.head()

--2025-04-21 06:43:17--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv’

ChatbotData.csv     100%[===================>] 868.99K  --.-KB/s    in 0.04s   

2025-04-21 06:43:17 (19.5 MB/s) - ‘ChatbotData.csv’ saved [889842/889842]

총 샘플 수: 11823


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [43]:
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import numpy as np

# 1. 데이터 불러오기
df = pd.read_csv('ChatbotData.csv')
df = df.dropna()  # 결측치 제거
df = df[['Q', 'A']]

questions = df['Q'].astype(str).tolist()
answers = df['A'].astype(str).tolist()

# 2. SubwordTextEncoder 훈련시키기
# 질문 + 답변 전체를 대상으로 빌드
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

# 특수 토큰 정의
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

# 최대 길이 설정
MAX_LENGTH = 40

# 3. 인코딩 함수
def encode(sentence):
    return START_TOKEN + tokenizer.encode(sentence) + END_TOKEN

# 4. 필터링 및 인코딩
input_sequences = []
output_sequences = []

for q, a in zip(questions, answers):
    q_encoded = encode(q)
    a_encoded = encode(a)

    if len(q_encoded) <= MAX_LENGTH and len(a_encoded) <= MAX_LENGTH:
        input_sequences.append(q_encoded)
        output_sequences.append(a_encoded)

# 5. 패딩
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(
    input_sequences, maxlen=MAX_LENGTH, padding='post')

output_tensor = tf.keras.preprocessing.sequence.pad_sequences(
    output_sequences, maxlen=MAX_LENGTH, padding='post')

input_tensor = np.array(input_tensor)
output_tensor = np.array(output_tensor)

print(f'전처리 완료. 총 샘플 수: {len(input_tensor)}')
print(f'VOCAB_SIZE: {VOCAB_SIZE}')
print(f'예시 질문 인코딩:', input_tensor[0])
print(f'예시 답변 인코딩:', output_tensor[0])

전처리 완료. 총 샘플 수: 11823
VOCAB_SIZE: 8172
예시 질문 인코딩: [8170 7909 4200 3054 7947 8171    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
예시 답변 인코딩: [8170 3837   71 7888 7960 8171    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [44]:
#모델정의

# 스케일드 닷 프로덕트 어텐션
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

# 멀티헤드 어텐션 레이어
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % num_heads == 0
        self.depth = d_model // num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])  # (batch, heads, seq_len, depth)

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)

        scaled_attention, _ = scaled_dot_product_attention(q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        return self.dense(concat_attention)

# 포지셔널 인코딩
def get_positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(
        10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / d_model)

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [45]:
# 포지션 와이즈 피드포워드
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        attn_output = self.mha(x, x, x, mask)
        out1 = self.layernorm1(x + self.dropout1(attn_output, training=training))

        ffn_output = self.ffn(out1)
        out2 = self.layernorm2(out1 + self.dropout2(ffn_output, training=training))
        return out2

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1 = self.mha1(x, x, x, look_ahead_mask)
        out1 = self.layernorm1(x + self.dropout1(attn1, training=training))

        attn2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        out2 = self.layernorm2(out1 + self.dropout2(attn2, training=training))

        ffn_output = self.ffn(out2)
        out3 = self.layernorm3(out2 + self.dropout3(ffn_output, training=training))
        return out3

In [46]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 input_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = get_positional_encoding(maximum_position_encoding, d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training=False, mask=None):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x) + self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x=x, training=training, mask=mask)  # 🔧 수정된 부분

        return x


class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 target_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = get_positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training=False, look_ahead_mask=None, padding_mask=None):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x) + self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.dec_layers[i](
                x=x,
                enc_output=enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask
            )  # 🔧 수정된 부분

        return x


class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super().__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                               input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                               target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):
        inp, tar = inputs

        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)

        enc_output = self.encoder(x=inp, training=training, mask=enc_padding_mask)
        dec_output = self.decoder(x=tar, enc_output=enc_output, training=training,
                                  look_ahead_mask=look_ahead_mask, padding_mask=dec_padding_mask)

        return self.final_layer(dec_output)

In [47]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

In [48]:
#하이퍼 파라미터 설정
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

EPOCHS = 100
BATCH_SIZE = 64

BUFFER_SIZE = 20000

In [49]:
# 전처리된 input_tensor, output_tensor로부터 dataset 생성
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, output_tensor))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [50]:
#손실함수 옵티마이져
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.96)

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [51]:
transformer = Transformer(
    num_layers=NUM_LAYERS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dff=DFF,
    input_vocab_size=VOCAB_SIZE,
    target_vocab_size=VOCAB_SIZE,
    pe_input=MAX_LENGTH,
    pe_target=MAX_LENGTH,
    rate=DROPOUT
)

checkpoint_path = "./checkpoints"
ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

# 체크포인트가 존재하면 로드
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print(' 체크포인트 복원됨')

 체크포인트 복원됨


In [52]:
#학습루프 정의

train_loss = tf.keras.metrics.Mean(name='train_loss')

@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    with tf.GradientTape() as tape:
        predictions = transformer(inputs=(inp, tar_inp), training=True)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

# 학습 시작
for epoch in range(EPOCHS):
    train_loss.reset_state()

    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)

    print(f"✅ Epoch {epoch+1}, Loss: {train_loss.result():.4f}")

    # 체크포인트 저장
    ckpt_save_path = ckpt_manager.save()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:1387: UserWarning: Layer 'transformer_2' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: '''tensorflow.python.framework.ops.EagerTensor' object has no attribute '_serialize_to_tensors'''
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'transformer_2', however the layer does not have a `build()` method implemented a

✅ Epoch 1, Loss: 6.3455
✅ Epoch 2, Loss: 5.6517
✅ Epoch 3, Loss: 5.4032
✅ Epoch 4, Loss: 5.2400
✅ Epoch 5, Loss: 5.1303
✅ Epoch 6, Loss: 5.0502
✅ Epoch 7, Loss: 4.9758
✅ Epoch 8, Loss: 4.9112
✅ Epoch 9, Loss: 4.8464
✅ Epoch 10, Loss: 4.7909
✅ Epoch 11, Loss: 4.7259
✅ Epoch 12, Loss: 4.6712
✅ Epoch 13, Loss: 4.6167
✅ Epoch 14, Loss: 4.5654
✅ Epoch 15, Loss: 4.5167
✅ Epoch 16, Loss: 4.4689
✅ Epoch 17, Loss: 4.4208
✅ Epoch 18, Loss: 4.3749
✅ Epoch 19, Loss: 4.3283
✅ Epoch 20, Loss: 4.2840
✅ Epoch 21, Loss: 4.2352
✅ Epoch 22, Loss: 4.1921
✅ Epoch 23, Loss: 4.1454
✅ Epoch 24, Loss: 4.0981
✅ Epoch 25, Loss: 4.0507
✅ Epoch 26, Loss: 4.0085
✅ Epoch 27, Loss: 3.9613
✅ Epoch 28, Loss: 3.9166
✅ Epoch 29, Loss: 3.8687
✅ Epoch 30, Loss: 3.8224
✅ Epoch 31, Loss: 3.7783
✅ Epoch 32, Loss: 3.7320
✅ Epoch 33, Loss: 3.6846
✅ Epoch 34, Loss: 3.6390
✅ Epoch 35, Loss: 3.5918
✅ Epoch 36, Loss: 3.5403
✅ Epoch 37, Loss: 3.4962
✅ Epoch 38, Loss: 3.4465
✅ Epoch 39, Loss: 3.3961
✅ Epoch 40, Loss: 3.3522
✅ Epoch 4

In [53]:
def evaluate(sentence):
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    sentence = tf.expand_dims(sentence, axis=0)  # (1, 문장길이)
    output = tf.expand_dims(START_TOKEN, 0)      # (1, 1)

    for i in range(MAX_LENGTH):
        predictions = transformer(inputs=(sentence, output), training=False)
        predictions = predictions[:, -1:, :]  # (1, 1, vocab)

        # 🔧 샘플링 + 타입 캐스팅
        predicted_id = tf.random.categorical(predictions[0], num_samples=1)  # (1, 1)
        predicted_id = tf.cast(predicted_id, dtype=tf.int32)  # 🔥 여기 중요
        output = tf.concat([output, predicted_id], axis=-1)   # (1, t+1)

        if predicted_id[0][0].numpy() == END_TOKEN[0]:
            break

    return tf.squeeze(output, axis=0)  # (시퀀스,)

In [54]:
print(predict("안녕"))


print(predict("요즘 기분이 어때?"))


안녕하세요.
저는 고민이 없어요.


In [55]:
print(predict("내일 날씨는 어때?"))

놀이동산은 다 잘 정리해 보세요.


In [56]:
print(predict("오늘 헤어졌어"))

다음부터는 잘 이겨낼 수 있어요.



⸻

📌 현재 상태 요약
	•	답변이 뭔가 말은 계속 이어지는데 의미는 없음


⸻

🤔 왜 이런 현상이 발생하냐?

원인	설명
🤯 학습 Epoch 부족	아직 문장 끝맺는 법을 안 배움 → 엔드토큰 조건도 잘 안 지킴
🔁 샘플링의 temperature 없음	샘플링만 하면 무작위성만 늘고 품질은 안 좋아질 수 있음
🔡 Subword tokenizer만 쓰고, 문법 고려 안 됨	한국어 형태소 구조 무시하고 서브워드만 썼으니 말이 이상해질 수 있음
📉 Loss가 4대인데 아직 부족	적어도 3 이하로 떨어져야 문장 단위 응답이 자연스러워짐



⸻

✅ 개선 방향 (빠르게 자연스럽게 만들고 싶다면)

1. 🔥 샘플링 온도 조절 추가 (너무 랜덤하지 않게)

def evaluate(sentence, temperature=0.8):
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    sentence = tf.expand_dims(sentence, axis=0)
    output = tf.expand_dims(START_TOKEN, 0)

    for i in range(MAX_LENGTH):
        predictions = transformer(inputs=(sentence, output), training=False)
        predictions = predictions[:, -1:, :]  # (1, 1, vocab)

        # ✅ temperature 적용
        logits = predictions[0] / temperature
        predicted_id = tf.random.categorical(logits, num_samples=1)
        predicted_id = tf.cast(predicted_id, dtype=tf.int32)
        output = tf.concat([output, predicted_id], axis=-1)

        if predicted_id[0][0].numpy() == END_TOKEN[0]:
            break

    return tf.squeeze(output, axis=0)



⸻

2. 🧠 학습 데이터 줄이고 Epoch 늘리기 (ex. 3000개로 50 epoch 학습)

input_tensor = input_tensor[:3000]
output_tensor = output_tensor[:3000]
EPOCHS = 100



In [57]:
def evaluate(sentence, temperature=0.8):
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    sentence = tf.expand_dims(sentence, axis=0)
    output = tf.expand_dims(START_TOKEN, 0)

    for i in range(MAX_LENGTH):
        predictions = transformer(inputs=(sentence, output), training=False)
        predictions = predictions[:, -1:, :]  # (1, 1, vocab)

        # ✅ temperature 적용
        logits = predictions[0] / temperature
        predicted_id = tf.random.categorical(logits, num_samples=1)
        predicted_id = tf.cast(predicted_id, dtype=tf.int32)
        output = tf.concat([output, predicted_id], axis=-1)

        if predicted_id[0][0].numpy() == END_TOKEN[0]:
            break

    return tf.squeeze(output, axis=0)

In [58]:
print(predict("12시 땡!"))

철 들고 공부하세요.


In [59]:
print(predict("안녕?"))

좋겠어요.


In [70]:
print(predict("취업하기 싫어"))

자신의 자신에 집중하세요. 언제나 1순오세요.


In [69]:
print(predict("여자친구"))

세상은 넓고 사람은 당신의 추억의 어필해보세요.


In [68]:
print(predict("배고파"))

뭐 좀 챙겨드세요.


In [64]:
print(predict("밥 먹었어?"))

저는 어때요?


In [65]:
print(predict("짜증나"))

짜증날 때는 기분 푸는 음악 어때요?


In [66]:
print(predict("심심해"))

친구들과 연락해보세요.


In [67]:
print(predict("졸려"))

낮잠을 써보세요.


In [71]:
# Flask, ngrok 설치
!pip install flask-ngrok

# Flask 서버 실행 코드
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    user_input = data.get("message", "")

    # 🔥 예외 처리
    if not user_input:
        return jsonify({"response": "입력이 비어있습니다."})

    try:
        response = predict(user_input)
    except Exception as e:
        response = f"에러 발생: {str(e)}"

    return jsonify({"response": response})

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [ ]:
#서버 구축 및 플러터 연결

In [86]:
# ngrok 다운로드
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip

# ngrok 인증 토큰 설정 (한 번만)
!./ngrok config add-authtoken 2w1u5aF8st2GpVBCUfQ80szZGwK_81UVsb3RtXZqHHtrJsE6C

Archive:  ngrok.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [93]:
%%writefile server.py
from flask import Flask, request, jsonify
from threading import Thread

app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    message = data.get("message", "")
    try:
        from google.colab import output
        result = predict(message)
    except Exception as e:
        result = f"에러: {str(e)}"
    return jsonify({"response": result})

if __name__ == "__main__":
    app.run(port=5000)

Overwriting server.py


In [94]:
import subprocess
import time

# Flask 서버 백그라운드에서 실행
flask_process = subprocess.Popen(["python3", "server.py"])
time.sleep(3)  # 서버 뜰 때까지 잠깐 대기

In [95]:
import subprocess
import time

# ngrok 백그라운드 실행
ngrok_process = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(3)  # 잠깐 기다렸다가

In [96]:
import requests

try:
    r = requests.get('http://localhost:4040/api/tunnels')
    tunnels = r.json()['tunnels']
    for tunnel in tunnels:
        print("📡 NGROK URL:", tunnel['public_url'])
except Exception as e:
    print("❌ ngrok 주소 불러오기 실패:", str(e))

📡 NGROK URL: https://0545-34-82-106-133.ngrok-free.app
